In [80]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import time

In [81]:
# set up splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/travis/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


In [82]:
# visit web page
page_url = "https://www.federalreserve.gov/newsevents/pressreleases.htm"
browser.visit(page_url)
time.sleep(10)

press_release_list = []

# iterate through all pages
for i in range(10):
    
    # create html object and scrape into soup
    html = browser.html
    soup = bs(html, "html.parser")

    press_releases = soup.find_all("div", "col-xs-9 col-md-10 eventlist__event")

    for release in press_releases:

        release_url = 'https://www.federalreserve.gov' + release.a['href']
        title = release.a.text.strip()
        category = release.b.text.strip()

        release_dict = {"url": release_url,
                        "title": title,
                        "category": category
                       }

        press_release_list.append(release_dict)
    
    # click the 'Next' button on each page
    try:
        browser.links.find_by_partial_text('Next').click()
          
    except:
        print("scraping complete")

In [83]:
len(press_release_list)

200

In [94]:
press_release_list[0]

{'url': 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20210625a.htm',
 'title': 'Federal Reserve Board announces it will extend for a final time its Paycheck Protection Program Liquidity Facility, or PPPLF, by an additional month to July 30, 2021',
 'category': 'Monetary Policy'}

In [90]:
press_release_details = []

for i in range(len(press_release_list)):
    link = press_release_list[i]["url"]
    
    # visit web page
    browser.visit(link)
    #time.sleep(1)

    # create html object and scrape into soup
    html = browser.html
    soup = bs(html, "html.parser")
    
    release_date = soup.find("div", class_="heading col-xs-12 col-sm-8 col-md-8").find("p", class_="article__time").text.strip()
    release_time = soup.find("div", class_="heading col-xs-12 col-sm-8 col-md-8").find("p", class_="releaseTime").text.strip()

    if release_time.endswith('EDT') or release_time.endswith('EST'):
        release_time = release_time[-14:].strip()
        release_time = release_time.replace(".", "")
    else:
        release_time = None

    paragraphs = soup.find("div", class_="col-xs-12 col-sm-8 col-md-8").find_all("p")
    text = ""
    for p in paragraphs:
        if len(text) == 0:
            text = p.text.strip()
        else:
            text = text + "\n" + p.text.strip()
    
    details_dict = {"url": link,
                    "date": release_date,
                    "time": release_time,
                    "text": text
                   }
    
    press_release_details.append(details_dict)


In [91]:
len(press_release_details)

200

In [96]:
press_release_details[0]

{'url': 'https://www.federalreserve.gov/newsevents/pressreleases/monetary20210625a.htm',
 'date': 'June 25, 2021',
 'time': '3:30 pm EDT',
 'text': "The Federal Reserve Board on Friday announced it will extend for a final time its Paycheck Protection Program Liquidity Facility, or PPPLF, by an additional month to July 30, 2021. The extension is being made as an operational accommodation to allow additional processing time for banks, community development financial institutions, and other financial institutions to pledge to the facility any Paycheck Protection Program, or PPP, loans approved by the Small Business Administration through the June 30 expiration of the PPP program.\nThe PPPLF extends term credit to financial institutions making PPP loans, accepting the PPP loans as collateral. The liquidity provided by the PPPLF bolsters the effectiveness of the PPP by helping eligible financial institutions fund PPP loans.\nThe PPPLF was established in April 2020 under the Board's 13(3) au